In [6]:
pip install opencv-contrib-python
# pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import numpy as np
import math

# ---------------- functions used for CV ----------------

# Compute center of the ArUco marker 
def compute_center(top_left,bottom_right):
    center_x = int((top_left[0] + bottom_right[0]) / 2.0)
    center_y = int((top_left[1] + bottom_right[1]) / 2.0)
    return center_x,center_y
    
# Compute angle of vector = (bottom_left -> top_left) w.r.t horizontal axis 
def compute_angle(top_left,bottom_left):
    x_diff = int(top_left[0] - bottom_left[0]) 
    y_diff = int(top_left[1] - bottom_left[1]) 
    return math.atan2(y_diff,x_diff)

def detect_red_polygons(frame,
                        min_area=300,       # filter tiny blobs
                        eps_factor=0.02):   # approximation factor for vertices
    """
    Returns:
      polygons: list of (N,2) int arrays with polygon vertices in image coords
      mask: binary mask used to detect red regions
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Red wraps around 0°, so use two ranges
    lower_red1 = np.array([0,   80,  80])
    upper_red1 = np.array([10,  255, 255])
    lower_red2 = np.array([170, 80,  80])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask  = cv2.bitwise_or(mask1, mask2)

    # Cleanup (optional but usually helpful)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=2)

    # Contours --> polygons
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue

        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps_factor * peri, True)  # (N,1,2)
        verts = approx.reshape(-1, 2)                            # (N,2)
        polygons.append(verts.astype(int))

    return polygons, mask

def detect_red_polygons_in_map(frame, p_tl, p_br,min_area=300, eps_factor=0.02):
    x0, y0 = p_tl
    x1, y1 = p_br

    # Ensure proper ordering
    x0, x1 = sorted((x0, x1))
    y0, y1 = sorted((y0, y1))

    roi = frame[y0:y1, x0:x1]

    polygons_roi, mask_roi = detect_red_polygons(roi,min_area=min_area,eps_factor=eps_factor)
    # Shift vertices back to full-frame coordinates
    polygons_full = []
    offset = np.array([[x0, y0]])
    for verts in polygons_roi:
        polygons_full.append(verts + offset)
    
    return polygons_full, mask_roi

# map a single image point (u,v) to world (X,Y)
def img_to_world(u,v,H):
    uv1 = np.array([u,v, 1.0], dtype=np.float32) # z = 1
    XY1 = H @ uv1
    XY1 /= XY1[2]  # scale back to world coordinates where z = 1
    return float(XY1[0]), float(XY1[1])

# helper function to display text
def putText(frame, text, x, y):
    cv2.putText(frame,str(text),(int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 255, 0),2)
    
# ---------------- constants used for CV ----------------

# Real-world coordinates of the 8 corners of the aruco markers
# Order must match the corner order from ArUco: TL, TR, BR, BL.
# Top-left marker 
pts_world_TL = np.array([
    [-6.5, -6.5],   # top-left
    [ 0.0, -6.5],   # top-right
    [ 0.0,  0.0],   # bottom-right
    [-6.5,  0.0],   # bottom-left
], dtype=np.float32)

# Bottom-right marker
pts_world_BR = np.array([
    [44.0, 28.0],   # top-left
    [51.0, 28.0],   # top-right
    [51.0, 35.0],   # bottom-right
    [44.0, 35.0],   # bottom-left
], dtype=np.float32)

pts_world = np.vstack([pts_world_TL, pts_world_BR])     # shape (8,2)

In [9]:
import cv2
import numpy as np

# ---------------- Main CV implementation ----------------

# ArUco dictionary (DICT_4X4_50) 
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters() 

# Open the webcam 
cap = cv2.VideoCapture(0)   # /dev/video0

if not cap.isOpened():
    print("Could not open webcam")
    exit()

while True:
    ret, frame = cap.read() # Frame is a numpy array of shape (height, width, 3) of type uint8 -> 0 to 255
    if not ret:
        print("Frame not received")
        break
    # Convert to grayscale (Aruco works on grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # gray is an array of shape (heigh, width, 1) type uint8
    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params) 
    
    marker0_TL = marker1_BR = marker2_thymio = marker3_goal = 0

    # ---------------- Marker detection and map defintion  ----------------
    if ids is not None and len(ids) >= 2:
        ids_flat = ids.flatten()
        
        for i, marker_id in enumerate(ids_flat):
            if marker_id == 0:
                marker0_TL = i
            elif marker_id == 1:
                marker1_BR = i
            elif marker_id == 2:
                marker2_thymio = i
            elif marker_id == 3:
                marker3_goal = i
        
        x0, y0 = corners[marker0_TL][0][2] # TL corner of map (2 = BR corner of marker0)
        x1, y1 = corners[marker1_BR][0][0] # BR corner of map (0 = TL corner of mareker1)
        
        # Four corners of the map 
        p_tl = (int(x0), int(y0))  
        p_br = (int(x1), int(y1))  
        p_tr = (int(x1), int(y0))
        p_bl = (int(x0), int(y1))
        
        # ---------------- Position/orientation of thymio and goal in image space ----------------
        thymio_x_img,thymio_y_img = compute_center(corners[marker2_thymio][0][0],corners[marker2_thymio][0][2]) 
        goal_x_img,goal_y_img = compute_center(corners[marker3_goal][0][0],corners[marker3_goal][0][2])
        thymio_theta = compute_angle(corners[marker2_thymio][0][0],corners[marker2_thymio][0][3]) # TL,BL
            
        # ---------------- Homography: image -> world ----------------
        # Image/pixel coordinates of the 8 corners
        pts_img_TL = corners[marker0_TL][0].astype(np.float32)  # shape (4,2)
        pts_img_BR = corners[marker1_BR][0].astype(np.float32)  # shape (4,2)
        pts_img = np.vstack([pts_img_TL, pts_img_BR]).astype(np.float32)    # shape (8,2)

        # print(type(pts_img), pts_img.shape, pts_img.dtype) # DEBUG
        # print(type(pts_world), pts_world.shape, pts_world.dtype) # DEBUG

        # Homography matrix (outliers = None since we use DLT)
        H, inliers = cv2.findHomography(pts_img, pts_world) # method=0: direct linear transform (DLT)
    
        # Compute position of thymio / goal / obstacles in real world (origin at top left corner of map)
        thymio_x, thymio_y = img_to_world(thymio_x_img, thymio_y_img, H)
        goal_x, goal_y = img_to_world(goal_x_img, goal_y_img, H)
    

        # ---------------- Polygon vertices detection ----------------
        polygons, red_mask_roi = detect_red_polygons_in_map(frame, p_tl, p_br)
        
        # Map to real world positions
        for verts in polygons: # verts: (N,2) in image coords
            cnt = verts.reshape(-1, 1, 2).astype(np.float32) # cv2 uses (N,1,2) format
            cnt_world = cv2.perspectiveTransform(cnt, H)  # same thing as img_to_world but for 2D array
            verts_world = cnt_world.reshape(-1, 2)        # (N,2) world coords
                
        #  ---------------- Display ----------------
        
        # Draw virtual map 
        cv2.line(frame, p_tl, p_tr, (0, 255, 0), 2)
        cv2.line(frame, p_tr, p_br,  (0, 255, 0), 2)
        cv2.line(frame, p_br,  p_bl, (0, 255, 0), 2)
        cv2.line(frame, p_bl, p_tl,  (0, 255, 0), 2)
        
        cv2.aruco.drawDetectedMarkers(frame, corners, ids) # TEST 
        
        # Thymio position / angle 
        cv2.putText(frame, str(round(thymio_theta,2)), (thymio_x_img,thymio_y_img),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 2)
        putText(frame, f"{thymio_theta:.2f}", thymio_x_img, thymio_y_img)
        putText(frame, f"({thymio_x:.2f}, {thymio_y:.2f})", thymio_x_img, thymio_y_img + 15) # +15 to move a bit down
    
        # Goal and obstacles positions
        putText(frame, f"({goal_x:.2f}, {goal_y:.2f})", goal_x_img, goal_y_img)
    
        # Obstacles (polygon[i] = array of vertices of the i-th polygon)
        for verts in polygons:
            cv2.polylines(frame, [verts], True, (255, 0, 255), 2) # connects all vertices together to form polygon
            for (x, y) in verts:
                cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1) # circle around each vertex of polygon

    # Show the frame
    cv2.imshow("Aruco Feed", frame)

    # Quit with q 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cap.release()
cv2.destroyAllWindows()


[ WARN:0@448.215] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
